# Imports

In [3]:
import random
import numpy as np
from collections import namedtuple
import time

# Game Environment

In [4]:
class Game:
    def __init__(self, initial_state):
        self.initial = initial_state

    def play(self, players):
        state = self.initial
        while True:
            for player in players:
                if self.is_terminal(state): return
                move = player(self, state)
                state = self.result(state, move)
                self.display(state)

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, move):
        raise NotImplementedError

    def utility(self, state, player):
        raise NotImplementedError

    def is_terminal(self, state):
        return NotImplementedError

    def display(self, state):
        return NotImplementedError

GameState = namedtuple('GameState', ['to_move', 'utility', 'board', 'moves'])

class TicTacToe(Game):
    def __init__(self, h=3, v=3, k=3):
        """Initialize TicTacToe with board size and winning condition."""
        super().__init__(GameState(to_move='X', utility=0, board={},
                                   moves=self._all_possible_moves(h, v)))
        self.h = h
        self.v = v
        self.k = k

    def _all_possible_moves(self, h, v):
        """Generate all possible moves on the given board size."""
        return [(x, y) for x in range(1, h + 1) for y in range(1, v + 1)]

    def actions(self, state):
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        next_player = 'O' if state.to_move == 'X' else 'X'
        return GameState(to_move=next_player,
                        utility=self.compute_utility(board, move, state.to_move),
                        board=board, moves=moves)

    def utility(self, state, player):
        return state.utility if player == 'X' else -state.utility

    def is_terminal(self, state):
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else - 1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

# MinMax Search

In [ ]:
MAXVALUE = np.inf
MINVALUE = -np.inf
def minmax_search(game, state):
    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = MINVALUE

        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v
    
    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = MAXVALUE

        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v
    
    print(f"Turn of {player}, available actions {game.actions(state)}")
    return max(game.actions(state), key = lambda a: min_value(game.result(state, a)))

def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algo):
    return lambda game, state:search_algo(game, state)

In [6]:
start_t = time.time()
TicTacToe().play([player(minmax_search), random_player])
end_t = time.time()
print(f"Execution time: {end_t-start_t:.2f} seconds")

Turn of X, available actions [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
X . . 
. . . 
. . . 
X . . 
. . . 
O . . 
Turn of X, available actions [(1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 2), (3, 3)]
X X . 
. . . 
O . . 
X X . 
. . . 
O O . 
Turn of X, available actions [(1, 3), (2, 1), (2, 2), (2, 3), (3, 3)]
X X X 
. . . 
O O . 
Execution time: 1.07 seconds


# AlphaBeta Pruning

In [7]:
MAXVALUE = np.inf
MINVALUE = -np.inf
def alphabeta_search(game, state):
    player = state.to_move

    def max_value(state, alpha = MINVALUE, beta = MAXVALUE):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = MINVALUE

        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v
    
    def min_value(state, alpha = MINVALUE, beta = MAXVALUE):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = MAXVALUE

        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v
    
    print(f"Turn of {player}, available actions {game.actions(state)}")
    return max(game.actions(state), key = lambda a: min_value(game.result(state, a)))

def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algo):
    return lambda game, state:search_algo(game, state)

In [8]:
start_t = time.time()
TicTacToe().play([player(alphabeta_search), random_player])
end_t = time.time()
print(f"Execution time: {end_t-start_t:.2f} seconds")

Turn of X, available actions [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
X . . 
. . . 
. . . 
X . . 
. . . 
. . O 
Turn of X, available actions [(1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2)]
X . X 
. . . 
. . O 
X . X 
. O . 
. . O 
Turn of X, available actions [(1, 2), (2, 1), (2, 3), (3, 1), (3, 2)]
X X X 
. O . 
. . O 
Execution time: 0.07 seconds


# ExpectiMax

In [9]:
MINVALUE = -np.inf
def expectimax_search(game, state):
    player = state.to_move

    def exp_value(state):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = 0

        for a in game.actions(state):
            p = 1/len(game.actions(state))
            v += p * max_value(game.result(state, a))

        return v

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = MINVALUE

        for a in game.actions(state):
            v = max(v, exp_value(game.result(state, a)))
        return v
    
    print(f"Turn of {player}, available actions {game.actions(state)}")
    return max(game.actions(state), key = lambda a: exp_value(game.result(state, a)))

def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algo):
    return lambda game, state:search_algo(game, state)

In [10]:
start_t = time.time()
TicTacToe().play([player(expectimax_search), random_player])
end_t = time.time()
print(f"Execution time: {end_t-start_t:.2f} seconds")

Turn of X, available actions [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]


X . . 
. . . 
. . . 
X . . 
. . O 
. . . 
Turn of X, available actions [(1, 2), (1, 3), (2, 1), (2, 2), (3, 1), (3, 2), (3, 3)]
X . X 
. . O 
. . . 
X . X 
. . O 
O . . 
Turn of X, available actions [(1, 2), (2, 1), (2, 2), (3, 2), (3, 3)]
X X X 
. . O 
O . . 
Execution time: 1.06 seconds


# AlphaBeta ExpectiMax

In [11]:
MAXVALUE = np.inf
MINVALUE = -np.inf
def abexpectimax_search(game, state):
    player = state.to_move

    def exp_value(state, alpha = MINVALUE, beta = MAXVALUE):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = 0

        for a in game.actions(state):
            p = 1/len(game.actions(state))
            v += p * max_value(game.result(state, a), alpha, beta)

            if v <= alpha:
                return v
            beta = min(v, beta)
        return v

    def max_value(state, alpha = MINVALUE, beta = MAXVALUE):
        if game.is_terminal(state):
            return game.utility(state, player)
        
        v = MINVALUE

        for a in game.actions(state):
            v = max(v, exp_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(v, alpha)
        return v
    
    print(f"Turn of {player}, available actions {game.actions(state)}")
    return max(game.actions(state), key = lambda a: exp_value(game.result(state, a)))

def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algo):
    return lambda game, state:search_algo(game, state)

In [12]:
start_t = time.time()
TicTacToe().play([player(abexpectimax_search), player(abexpectimax_search)])
end_t = time.time()
print(f"Execution time: {end_t-start_t:.2f} seconds")

Turn of X, available actions [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
X . . 
. . . 
. . . 
Turn of O, available actions [(1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
X . . 
. . O 
. . . 
Turn of X, available actions [(1, 2), (1, 3), (2, 1), (2, 2), (3, 1), (3, 2), (3, 3)]
X . X 
. . O 
. . . 
Turn of O, available actions [(1, 2), (2, 1), (2, 2), (3, 1), (3, 2), (3, 3)]
X O X 
. . O 
. . . 
Turn of X, available actions [(2, 1), (2, 2), (3, 1), (3, 2), (3, 3)]
X O X 
. . O 
X . . 
Turn of O, available actions [(2, 1), (2, 2), (3, 2), (3, 3)]
X O X 
O . O 
X . . 
Turn of X, available actions [(2, 2), (3, 2), (3, 3)]
X O X 
O X O 
X . . 
Execution time: 0.04 seconds


# MCTS

In [17]:
class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states."""

    def __init__(self, parent=None, state=None, U=0, N=0):
        # Initialize the MCT_Node with parent, state, U (utility), and N (visit count)
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {}  # Dictionary to store child nodes
        self.actions = None  # Placeholder for actions associated with this node


def ucb(n, C=1.4):
    """
    Upper Confidence Bound (UCB) function for node selection.
    Args:
        n: The node for which to calculate UCB.
        C: Exploration constant (default value is 1.4).

    Returns:
        UCB value for the given node.
    """
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N)


def monte_carlo_tree_search(game, state, N=5):
    """
    Monte Carlo Tree Search (MCTS) algorithm.
    Args:
        game: The game environment (e.g., chess, tic-tac-toe).
        state: The current state of the game.
        N: Number of iterations (default value is 3).

    Returns:
        The best child node based on MCTS.
    """
    def select(n):
        """Select a leaf node in the tree."""
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """Expand the leaf node by adding all its children states."""
        if not n.children and not game.is_terminal(n.state):
            n.children = {MCT_Node(state=game.result(n.state, action), parent=n): action
                          for action in game.actions(n.state)}
        return select(n)

    def simulate(game, state):
        """
        Simulate the utility of the current state by randomly picking a step.
        Args:
            game: The game environment.
            state: The current state.

        Returns:
            Utility value (negative because we want to maximize utility).
        """
        player = state.to_move
        while not game.is_terminal(state):
            print(f"Simulating turn of {state.to_move}, available actions {game.actions(state)}")
            action = random.choice(list(game.actions(state)))
            state = game.result(state, action)
        v = game.utility(state, player)
        return -v   #nodes contain the utility for the opposite player

    def backprop(n, utility):
        """Pass the utility back to all parent nodes."""
        if utility > 0:
            n.U += utility
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility)

    root = MCT_Node(state=state)  # Create the root node

    for _ in range(N):
        leaf = select(root)  # Select a leaf node
        child = expand(leaf)  # Expand the leaf node
        result = simulate(game, child.state)  # Simulate utility
        backprop(child, result)  # Backpropagate utility

    print(f"Turn of {state.to_move}, available actions {game.actions(state)}")
    max_state = max(root.children, key=lambda p: p.N)  # Choose the best child node

    return root.children.get(max_state)  # Return the best child node

In [18]:
start_t = time.time()
TicTacToe().play([player(monte_carlo_tree_search),random_player])
end_t = time.time()
print(f"Execution time: {end_t-start_t:.2f} seconds")

Simulating turn of O, available actions [(1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
Simulating turn of X, available actions [(1, 2), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
Simulating turn of O, available actions [(1, 2), (2, 1), (2, 2), (3, 1), (3, 2), (3, 3)]
Simulating turn of X, available actions [(2, 1), (2, 2), (3, 1), (3, 2), (3, 3)]
Simulating turn of O, available actions [(2, 1), (3, 1), (3, 2), (3, 3)]
Simulating turn of X, available actions [(2, 1), (3, 2), (3, 3)]
Simulating turn of X, available actions [(1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
Simulating turn of O, available actions [(1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2)]
Simulating turn of X, available actions [(1, 3), (2, 2), (2, 3), (3, 1), (3, 2)]
Simulating turn of O, available actions [(1, 3), (2, 2), (2, 3), (3, 1)]
Simulating turn of X, available actions [(1, 3), (2, 2), (3, 1)]
Simulating turn of X, available actions [(1, 2), (2, 1), (2, 2), (2, 3), (3, 1), (3, 